In [1]:
import geopandas as gpd
import pandas as pd

# Carregar dados
df = gpd.read_file("..\..\Data\Processed\PT-FireSprd_v2.1\L2_FireBehavior\PT-FireProg_v2.1_L2_final.shp")

# Lista de estatísticas para cada tipo de coluna
stats_config = {
    'numeric': ['mean', 'median', 'std', 'min', 'max', 'count'],
    'datetime': ['min', 'max'],
    'object': ['nunique', 'mode']
}

# Função para processar todos os grupos
all_results = []
for fid, group in df.groupby('fid'):
    group_result = {'fid': fid, 'total_features': len(group)}
    
    for col in group.columns:
        if col in ['geometry', 'fid']:
            continue
            
        # Estatísticas básicas
        group_result[f'{col}_count'] = len(group[col])
        group_result[f'{col}_nonnull'] = group[col].count()
        
        # Por tipo de dado
        if pd.api.types.is_numeric_dtype(group[col]):
            for stat in stats_config['numeric']:
                if stat == 'count':
                    continue
                try:
                    group_result[f'{col}_{stat}'] = getattr(group[col], stat)()
                except:
                    group_result[f'{col}_{stat}'] = None
        
        elif pd.api.types.is_datetime64_any_dtype(group[col]):
            for stat in stats_config['datetime']:
                try:
                    group_result[f'{col}_{stat}'] = getattr(group[col], stat)()
                except:
                    group_result[f'{col}_{stat}'] = None
        
        elif pd.api.types.is_string_dtype(group[col]) or pd.api.types.is_object_dtype(group[col]):
            for stat in stats_config['object']:
                if stat == 'nunique':
                    group_result[f'{col}_nunique'] = group[col].nunique()
                elif stat == 'mode':
                    mode_val = group[col].mode()
                    group_result[f'{col}_mode'] = mode_val.iloc[0] if not mode_val.empty else None
    
    all_results.append(group_result)

# Criar DataFrame final
result_df = pd.DataFrame(all_results)

# Salvar
result_df.to_csv("..\..\Data\Data_Exploration\Fire_stats_all_vars.csv", index=False)
print(f"Análise completa salva. {len(result_df)} grupos processados.")
print(f"Total de métricas: {len(result_df.columns)}")

c:\Users\dias3\miniconda3\envs\geoenv\Lib\site-packages\pyogrio\core.py:35: RuntimeWarning: Could not detect GDAL data files.  Set GDAL_DATA environment variable to the correct path.
  _init_gdal_data()
c:\Users\dias3\miniconda3\envs\geoenv\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\dias3\miniconda3\envs\geoenv\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\dias3\miniconda3\envs\geoenv\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\dias3\miniconda3\envs\geoenv\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\dias3\miniconda3\envs\geoenv\Lib\site-packages\numpy\lib\nanfunc

Análise completa salva. 164 grupos processados.
Total de métricas: 810
